In [28]:
from __future__ import print_function

import pandas as pd
import numpy as np
import joblib as jl

from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LogisticRegressionCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_validate
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.model_selection import learning_curve
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, Normalizer, MinMaxScaler
from sklearn.preprocessing import FunctionTransformer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.decomposition import PCA
from sklearn.manifold import Isomap, TSNE
from sklearn import metrics
from sklearn.feature_selection import chi2
from tqdm import tqdm

from mpl_toolkits.mplot3d import Axes3D
import matplotlib.pyplot as plt
%matplotlib inline

root = 'naive_clf_assessment'

# Load data

In [3]:
labels = pd.read_csv('../../tmp/1_METONLY_vs_METX/matched_CEM_table.csv', header=0).rename({'Unnamed: 0': 'PIN'}, axis=1)[['PIN', 'CLASS']].set_index('PIN')
print(labels.shape)
print(labels.head())

(11744, 1)
         CLASS
PIN           
568483       1
923748       0
2035328      0
2239386      1
2244533      1


In [6]:
data = pd.read_csv('../../tmp/raw_sequences.csv', header=0, index_col=0).loc[labels.index, 'seq']
data.head()

PIN
568483     10990 0 23 1 725 0 10990 0 10990 0 10990 0 109...
923748     66653 0 65070 0 66515 0 10962 0 105 0 56807 2 ...
2035328    66536 0 74995 0 66512 0 66551 0 66608 0 66716 ...
2239386    10991 0 23 3 73928 0 66560 0 66515 0 66551 0 6...
2244533    10990 0 23 2 23 0 10990 0 73928 0 66500 0 6656...
Name: seq, dtype: object

In [50]:
D = 100
embedding = pd.read_csv('../../tmp/embedding.{}d.csv'.format(D), header=0, index_col=0)
embedding.head()

,0,1,2,3,4,5,6,7,8,9,...,90,91,92,93,94,95,96,97,98,99
1,-0.029115,0.049347,0.013910,0.101860,-0.072991,0.096457,0.019702,0.136783,-0.079544,-0.018860,...,0.027578,-0.046793,0.043735,0.005530,-0.098460,0.031738,0.038261,-0.032858,0.111000,-0.017188
2,-0.029115,0.049347,0.013910,0.101860,-0.072991,0.096457,0.019702,0.136783,-0.079544,-0.018860,...,0.027578,-0.046793,0.043735,0.005530,-0.098460,0.031738,0.038261,-0.032858,0.111000,-0.017188
3,-0.060755,0.045741,0.062243,0.069529,-0.040196,0.045210,-0.023885,0.056138,-0.049073,-0.011440,...,-0.013475,-0.031083,0.013416,0.017287,0.001561,0.072467,-0.025701,-0.117037,0.117738,0.028410
4,-0.016764,0.093411,-0.027355,0.094497,-0.053251,0.091619,0.013388,0.134238,0.001519,0.040661,...,0.000044,-0.100772,0.119578,-0.027747,-0.119778,0.095200,0.027981,-0.011990,0.172747,-0.001647
57350,-0.086966,0.077627,-0.019118,0.014159,0.061399,0.043119,-0.011132,0.105824,-0.036162,0.047998,...,-0.043025,0.018667,-0.026304,0.035568,-0.006327,0.016930,0.015058,-0.148772,0.094491,-0.039460


# Prepare embedding dataset

In [53]:
# dfx_dump = {}
# dfx_dump[50] = dfX
# dfx_dump[100] = dfX

In [51]:
X = []
for idx in tqdm(data.index):
    seq = map(int, data.loc[idx].split(' ')[::2])
    x_i = np.array([embedding.loc[i] for i in seq])
    X.append(np.sum(x_i, axis=0))
dfX = pd.DataFrame(data=np.array(X), index=data.index)

100%|██████████| 11744/11744 [05:27<00:00, 35.88it/s]


In [52]:
dfX.head(2)

,0,1,2,3,4,5,6,7,8,9,...,90,91,92,93,94,95,96,97,98,99
PIN,,,,,,,,,,,,,,,,,,,,,
568483,-7.880987,4.687762,5.087566,9.196922,1.682410,2.517551,-5.620493,13.286569,-3.615799,2.949311,...,-2.997009,-6.846030,-3.424541,6.260076,-11.227638,3.159536,4.645165,-10.581501,22.922366,0.289747
923748,-16.991643,11.400465,9.682103,15.076172,-1.438836,14.142061,-3.386783,32.498087,-15.229934,6.553371,...,-2.308732,-20.690658,4.520073,8.510967,-25.070134,26.194851,7.711837,-28.861742,54.753652,-1.466212


# Evaluate simple pipeline

In [54]:
pipe = Pipeline([
    ('clf', LogisticRegression())
    ])

In [55]:
sss = StratifiedShuffleSplit(n_splits=5, test_size=0.25)

cv_results_ = cross_validate(pipe, dfX, labels.values.ravel(), n_jobs=-1,
                             cv=sss, scoring=('accuracy', 'precision',
                                              'recall', 'f1', 'roc_auc'),
                             return_train_score=True
                            )

In [56]:
cv_results = pd.DataFrame(cv_results_).transpose()
cv_results.loc[:, 'MEAN'] = cv_results.mean(axis=1)
cv_results.loc[:, 'STD'] = cv_results.std(axis=1)
out = cv_results[['MEAN', 'STD']]
out

,MEAN,STD
fit_time,2.161930,0.695576
score_time,0.038633,0.015089
test_accuracy,0.678883,0.007836
test_f1,0.665904,0.012149
test_precision,0.693693,0.003964
test_recall,0.640463,0.019618
test_roc_auc,0.716391,0.007955
train_accuracy,0.686580,0.005477
train_f1,0.674256,0.005133
train_precision,0.701905,0.006962


In [57]:
out.to_csv(root+'/d{}_cv_results.csv'.format(D))

# Confusion matrix

In [58]:
import itertools

def plot_confusion_matrix(cm, classes,
                          normalize=False,
                          title='Confusion matrix',
                          cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    fmt = '.2f' if normalize else 'd'
    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, format(cm[i, j], fmt),
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')
    return plt

In [59]:
y = labels.values.ravel()

sss = StratifiedShuffleSplit(n_splits=1, test_size=0.5)
tr_idx, ts_idx = next(sss.split(np.empty_like(y), y))

x_train = dfX.iloc[tr_idx]
x_test = dfX.iloc[ts_idx]
y_train, y_test = y[tr_idx], y[ts_idx]
print(x_train.shape)

classifier = LogisticRegressionCV(n_jobs=-1)
y_pred = classifier.fit(x_train, y_train).predict(x_test)

# Compute confusion matrix
cnf_matrix = metrics.confusion_matrix(y_test, y_pred)
np.set_printoptions(precision=2)

(5872, 100)


Test acc: 0.677
Test AUC: 0.677


In [62]:
print('Test acc: {:2.3f}'.format(metrics.accuracy_score(y_test, y_pred)))
print('Test AUC: {:2.3f}'.format(metrics.roc_auc_score(y_test, y_pred)))

# Plot non-normalized confusion matrix
plt.figure(dpi=100)
plot_confusion_matrix(cnf_matrix, classes=['METONLY', 'METX'],
                      title='Confusion matrix', cmap=plt.cm.Blues)
plt.savefig(root+'/d{}-_cm.png'.format(D))

# Learning curves

In [63]:
def plot_learning_curve(estimator, title, X, y, ylim=None, cv=None,
                        n_jobs=1, train_sizes=np.linspace(.1, 1.0, 5)):
    """
    Generate a simple plot of the test and training learning curve.
    """
    plt.figure(dpi=100)
    plt.title(title)
    if ylim is not None:
        plt.ylim(*ylim)
    
    plt.xlabel("Training examples")
    plt.ylabel("ROC AUC")
    
    train_sizes, train_scores, test_scores = learning_curve(
        estimator, X, y, cv=cv, n_jobs=n_jobs, train_sizes=train_sizes, scoring='roc_auc')
    
    train_scores_mean = np.mean(train_scores, axis=1)
    train_scores_std = np.std(train_scores, axis=1)
    test_scores_mean = np.mean(test_scores, axis=1)
    test_scores_std = np.std(test_scores, axis=1)

    plt.fill_between(train_sizes, train_scores_mean - train_scores_std,
                     train_scores_mean + train_scores_std, alpha=0.1,
                     color="C0")
    plt.fill_between(train_sizes, test_scores_mean - test_scores_std,
                     test_scores_mean + test_scores_std, alpha=0.1, color="C1")
    plt.plot(train_sizes, train_scores_mean, 'o-', color="C0",
             label="Training score")
    plt.plot(train_sizes, test_scores_mean, 'x-', color="C1",
             label="Cross-validation score")

    plt.legend(loc="best")
    plt.tight_layout()
    return plt

In [66]:
plot_learning_curve(pipe, 'D {}-gram'.format(D),
                    dfX, labels.values.ravel(), cv=sss, ylim=[0.5, 1],
                    n_jobs=-1, train_sizes=np.linspace(0.01, 1, 20))
plt.savefig(root+'/d{}_lcurves.png'.format(D));

# Data viz

In [69]:
pipe = Pipeline([
    ('dr1', TSNE(n_components=3))
    ])

x_r = pipe.fit_transform(dfX.values)

In [70]:
fig = plt.figure(dpi=300)
ax = fig.add_subplot(111, projection='3d')

idx = np.arange(x_r.shape[0])
np.random.shuffle(idx)
idx = idx[:600]

ax.scatter(x_r[idx, 0], x_r[idx, 1], x_r[idx, 2], c=y[idx]);